In [ ]:
from random import random
from numpy import array
from numpy import cumsum
from numpy import array_equal
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional

# create a cumulative sum sequence
def get_sequence(n_timesteps):
	# create a sequence of random numbers in [0,1]
	X = array([random() for _ in range(n_timesteps)])
	# calculate cut-off value to change class values
	limit = n_timesteps/2.0
	# determine the class outcome for each item in cumulative sequence
	y = array([0 if x < limit else 1 for x in cumsum(X)])
	return X, y

# create multiple samples of cumulative sum sequences
def get_sequences(n_sequences, n_timesteps):
	seqX, seqY = list(), list()
	# create and store sequences
	for _ in range(n_sequences):
		X, y = get_sequence(n_timesteps)
		seqX.append(X)
		seqY.append(y)
	# reshape input and output for lstm
	seqX = array(seqX).reshape(n_sequences, n_timesteps, 1)
	seqY = array(seqY).reshape(n_sequences, n_timesteps, 1)
	return seqX, seqY

# define problem
n_timesteps = 10

# define LSTM
model = Sequential()
model.add(Bidirectional(LSTM(50, return_sequences=True), input_shape=(n_timesteps, 1)))
model.add(TimeDistributed(Dense(1, activation='sigmoid')))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

# train LSTM
X, y = get_sequences(500, n_timesteps)
model.fit(X, y, epochs=2, batch_size=10)

# evaluate LSTM
X, y = get_sequences(10, n_timesteps)
loss, acc = model.evaluate(X, y, verbose=0)
print('Loss: %f, Accuracy: %f' % (loss, acc*100))

# make predictions
for _ in range(5):
	X, y = get_sequences(1, n_timesteps)
	yhat = model.predict_classes(X, verbose=0)
	exp, pred = y.reshape(n_timesteps), yhat.reshape(n_timesteps)
	print('y=%s, yhat=%s, correct=%s' % (exp, pred, array_equal(exp,pred)))